In [1]:
!uv add youtube-transcript-api

Resolved 151 packages in 0.52ms
Audited 146 packages in 0.98ms


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi
from openai import OpenAI
from pydantic import BaseModel
import pickle
import datetime

In [3]:
openai_client = OpenAI()

In [4]:
video_id = 'ph1PxZIkz1o'

In [5]:
#!wget https://github.com/alexeygrigorev/ai-bootcamp-codespace/raw/refs/heads/main/week1/ph1PxZIkz1o.bin

In [6]:
with open(f'{video_id}.bin', 'rb') as f_in:
    transcript = pickle.load(f_in)

In [7]:
transcript[:11]

[FetchedTranscriptSnippet(text='So hi everyone. Uh today we are going to', start=0.0, duration=5.04),
 FetchedTranscriptSnippet(text='talk about our upcoming course. The', start=2.96, duration=3.52),
 FetchedTranscriptSnippet(text='upcoming course is called machine', start=5.04, duration=5.92),
 FetchedTranscriptSnippet(text='learning zoom camp. And um this is', start=6.48, duration=5.92),
 FetchedTranscriptSnippet(text='already I put the link in the', start=10.96, duration=3.599),
 FetchedTranscriptSnippet(text="description. So if you're watching um", start=12.4, duration=4.719),
 FetchedTranscriptSnippet(text="this video in recording or you're", start=14.559, duration=4.88),
 FetchedTranscriptSnippet(text='watching it live, you go here in the', start=17.119, duration=4.561),
 FetchedTranscriptSnippet(text='description after under this video and', start=19.439, duration=5.6),
 FetchedTranscriptSnippet(text='then you see a link course. uh click on', start=21.68, duration=6.24),
 Fetche

In [8]:
def format_timestamp(seconds: float) -> str:
    """Convert seconds to H:MM:SS if > 1 hour, else M:SS"""
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"

In [9]:
def make_subtitles(transcript) -> str:
    lines = []

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n', ' ')
        lines.append(ts + ' ' + text)

    return '\n'.join(lines)


In [10]:
subtitles = make_subtitles(transcript)

In [11]:
print(subtitles[:500])

0:00 So hi everyone. Uh today we are going to
0:02 talk about our upcoming course. The
0:05 upcoming course is called machine
0:06 learning zoom camp. And um this is
0:10 already I put the link in the
0:12 description. So if you're watching um
0:14 this video in recording or you're
0:17 watching it live, you go here in the
0:19 description after under this video and
0:21 then you see a link course. uh click on
0:25 that link and this bring you will bring
0:27 you to
0:29 this website this GitHub


In [12]:
instructions = """
Summarize the transcript and describe the main purpose of the video
and the main ideas. 

Also output chapters with time. Use usual sentence case, not Title Case for the chapter.

Output format: 

<OUTPUT>
Summary

timestamp chapter 
timestamp chapter
...
timestamp chapter
</OUTPUT>
"""

In [13]:
def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

In [14]:
answer = llm(subtitles, instructions=instructions)
print(answer)

<OUTPUT>
Summary

The video discusses an upcoming course titled "Machine Learning Zoom Camp," aimed primarily at individuals interested in machine learning engineering. The host outlines the course structure, content updates, prerequisites, and the focus on deployment and practical skills rather than purely theoretical knowledge. Students are encouraged to ask questions through a linked platform, and the session aims to clarify details about the course's objectives, modules, and job readiness.

Main ideas include:
1. Course details and linkage.
2. The shift towards deploying machine learning models.
3. The value of project-based learning and real-world applications.
4. Prerequisites in programming and command-line usage.
5. Overview of job prospects post-course completion.

timestamp chapter 
0:00 introduction to machine learning zoom camp 
0:50 course structure and updates 
2:36 question about job placement 
4:20 depth of computer vision and framework choice 
6:02 prerequisites for th

In [15]:
class Chapter(BaseModel):
    timestamp: str
    title: str

class YTSummaryResponse(BaseModel):
    summary: str
    chapters: list[Chapter]

In [16]:
instructions = """
Summarize the transcript and describe the main purpose of the video
and the main ideas. 

Also output chapters with time. Use usual sentence case, not Title Case for the chapter.

More chapters is better than fewer chapters. Have a chapter at least every 3-5 minutes
""".strip()

In [17]:
messages = [
    {"role": "system", "content": instructions}, 
    {"role": "user", "content": subtitles}
]

In [18]:
response = openai_client.responses.parse(
    model='gpt-4o-mini',
    input=messages,
    text_format=YTSummaryResponse
)

In [19]:
summary = response.output[0].content[0].parsed

print(summary.summary)
print()
for c in summary.chapters:
    print(c.timestamp, c.title)

This video discusses the upcoming "Machine Learning Zoom Camp" course, highlighting its structure, content updates, and prerequisites. The instructor emphasizes its focus on machine learning engineering and deployment skills, responding to various viewer questions about job placements, course content, and suitable backgrounds for participants. They clarify that while the course provides foundational knowledge, it mainly aims to prepare students for entry-level roles in machine learning engineering rather than data science. The instructor also encourages using resources like GitHub and ChatGPT to aid learning.

0:00 Introduction to machine learning zoom camp
2:50 Course updates and content changes
5:00 Job placement opportunities
6:40 Deep learning and computer vision coverage
8:40 Prerequisites for the course
11:30 Focus on programming, not deep math knowledge
13:05 Target audience for ML engineers vs data scientists
14:35 Use of companion book for the course
17:01 Computer requirement

In [22]:
def llm_structured(instructions, user_prompt, output_format, model="gpt-4o-mini"):
    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": user_prompt}
    ]

    response = openai_client.responses.parse(
        model=model,
        input=messages,
        text_format=output_format
    )

    return response.output[0].content[0].parsed

In [23]:
summary = llm_structured(
    instructions=instructions,
    user_prompt=subtitles,
    output_format=YTSummaryResponse
)

In [24]:
summary

YTSummaryResponse(summary='The video outlines details about the upcoming Machine Learning Zoom Camp course, addressing various aspects such as course structure, content updates, prerequisites, and answering participant questions. It emphasizes that the course is aimed primarily at those wanting to become machine learning engineers rather than data scientists, though data scientists may find it beneficial. The instructor discusses the engineering focus of the curriculum, the hands-on approach to learning, the importance of coding skills, and the nature of projects and assignments in the course.', chapters=[Chapter(timestamp='0:00', title='Introduction to machine learning zoom camp'), Chapter(timestamp='0:47', title='Course details and start date'), Chapter(timestamp='1:29', title='Course content updates'), Chapter(timestamp='3:55', title='Job placement and outcomes'), Chapter(timestamp='5:36', title='Prerequisites for the course'), Chapter(timestamp='6:45', title='Using PyTorch and Tens